## Setup

In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random
import logging

In [ ]:
# Directory Setup
# Change to project directory
os.chdir("..")
# Input data directory
data_dir = "ag_news"
inputdirectory = Path(f"./data/raw/input/{data_dir}")
# This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data/interim/output/{out_dir}")

In [2]:
ag_news = pd.read_csv("data/raw/ag_news_train.csv", header=None)

In [3]:
# ag_news_txt = ag_news[2].str.cat(sep="\n")
ag_news_txt = ag_news[2].iloc[:50].str.cat(sep="\n")

In [4]:
with open("data/raw/input/ag_news/ag_news.txt", "w") as f:
    f.write(ag_news_txt)

## Load Documents

In [5]:
## Dir PDF Loader
# loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
# loader = PyPDFLoader("./data/MedicalDocuments/orf-path_health-n1.pdf")
loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.02it/s]

Number of chunks =  7
Interest rates are trimmed to 7.5 by the South African central bank,  but the lack of warning hits the rand and surprises markets.

The cost of buying both new and second hand cars fell sharply over the past five years, a new survey has found.

South Korea's central bank cuts interest rates by a quarter percentage point to 3.5 in a bid to drive growth in the economy.

An auction of shares in Google, the web search engine which could be floated for as much as \$36bn, takes place on Friday.

Hewlett-Packard shares fall after disappointing third-quarter profits, while the firm warns the final quarter will also fall short of expectations.

One of the oldest textile operators on the Indian Ocean island of Mauritius last week shut seven factories and cut 900 jobs.

Chad asks the IMF for a loan to pay for looking after more than 100,000 refugees from conflict-torn Darfur in western Sudan.

The company running the Japanese nuclear plant hit by a fatal accident is to close

## Create a dataframe of all the chunks

In [6]:
from llmgrapher.helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

2024-05-12 21:46:43.818 | INFO     | llmgrapher.config:<module>:11 - PROJ_ROOT path is: /home/kleptotrace/Projects/git/llmgrapher


(7, 3)


/home/kleptotrace/miniforge3/envs/knowledge-graph/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,text,source,chunk_id
0,"Reuters - Short-sellers, Wall Street's dwindli...",data/raw/input/ag_news/ag_news.txt,0c78cdf041964504a6c44558c9b0ecf1
1,Forbes.com - After earning a PH.D. in Sociolog...,data/raw/input/ag_news/ag_news.txt,56769f34d4624076b5d248e4f4a418c5
2,NEW YORK (Reuters) - The dollar tumbled broadl...,data/raw/input/ag_news/ag_news.txt,bca3e5bb7cd44bf9abf27c6877830af9
3,Interest rates are trimmed to 7.5 by the South...,data/raw/input/ag_news/ag_news.txt,ed4358991eb244f28f3515f8aece72ad
4,A group led by the UAE's Etisalat plans to spe...,data/raw/input/ag_news/ag_news.txt,407eb7dd096449bdb823fc632867d7a8


## Extract Concepts

In [7]:
## This function uses the helpers/prompt function to extract concepts from text
from llmgrapher.helpers.df_helpers import df2Graph
from llmgrapher.helpers.df_helpers import graph2Df

If regenerate is set to True then the dataframes are regenerated and Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of edges

        df = dataframe of chunks


Else the dataframes are read from the output directory

In [8]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model='zephyr:latest')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

100%|█████████████████████████████████████████████| 7/7 [00:35<00:00,  5.09s/it]

Possibly due to JSON Decode Error, 2 chunks have been skipped
(34, 5)


,node_1,node_2,edge,chunk_id,count
0,short-sellers,well-timed and occasionally controversial play...,"Carlyle Group, a private investment firm with ...",0c78cdf041964504a6c44558c9b0ecf1,4
1,soaring crude prices,intelligence showed a rebel militia could stri...,Authorities have halted oil export flows from ...,0c78cdf041964504a6c44558c9b0ecf1,4
2,tearaway world oil prices,offsetting a positive outlook from computer ma...,Stocks ended slightly higher on Friday but sta...,0c78cdf041964504a6c44558c9b0ecf1,4
3,assets of the nation's retail money market mut...,$36;849.98 trillion,Assets of the nation's retail money market mut...,0c78cdf041964504a6c44558c9b0ecf1,4
4,midsummer slump,indicating the economy is improving from a mid...,"Retail sales bounced back a bit in July, and n...",0c78cdf041964504a6c44558c9b0ecf1,4


## Calculating contextual proximity

In [9]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
743,us dollar,growth,"bca3e5bb7cd44bf9abf27c6877830af9,bca3e5bb7cd44...",2,contextual proximity
756,us trade deficit,economic recovery,"bca3e5bb7cd44bf9abf27c6877830af9,bca3e5bb7cd44...",2,contextual proximity
760,us trade deficit,growth,"bca3e5bb7cd44bf9abf27c6877830af9,bca3e5bb7cd44...",2,contextual proximity
773,value-focused mutual funds,economic recovery,"bca3e5bb7cd44bf9abf27c6877830af9,bca3e5bb7cd44...",2,contextual proximity
777,value-focused mutual funds,growth,"bca3e5bb7cd44bf9abf27c6877830af9,bca3e5bb7cd44...",2,contextual proximity


### Merge both the dataframes

In [10]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,ability to draw foreign capital,economic recovery,"bca3e5bb7cd44bf9abf27c6877830af9,bca3e5bb7cd44...",contextual proximity,2
1,ability to draw foreign capital,growth,"bca3e5bb7cd44bf9abf27c6877830af9,bca3e5bb7cd44...",contextual proximity,2
2,assets of the nation's retail money market mut...,$36;849.98 trillion,0c78cdf041964504a6c44558c9b0ecf1,Assets of the nation's retail money market mut...,4
3,back-to-school season,economic recovery,"bca3e5bb7cd44bf9abf27c6877830af9,bca3e5bb7cd44...",contextual proximity,2
4,back-to-school season,growth,"bca3e5bb7cd44bf9abf27c6877830af9,bca3e5bb7cd44...",contextual proximity,2
...,...,...,...,...,...
127,us dollar,growth,"bca3e5bb7cd44bf9abf27c6877830af9,bca3e5bb7cd44...",contextual proximity,2
128,us trade deficit,economic recovery,"bca3e5bb7cd44bf9abf27c6877830af9,bca3e5bb7cd44...",The record U.S. trade deficit in June cast fre...,6
129,us trade deficit,growth,"bca3e5bb7cd44bf9abf27c6877830af9,bca3e5bb7cd44...",contextual proximity,2
130,value-focused mutual funds,economic recovery,"bca3e5bb7cd44bf9abf27c6877830af9,bca3e5bb7cd44...",contextual proximity,2


## Calculate the NetworkX Graph

In [11]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(64,)

In [12]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

### Calculate communities for coloring the nodes

In [13]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  17
[['$36;849.98 trillion', "assets of the nation's retail money market mutual funds"], ['ability to draw foreign capital', 'back-to-school season', 'domestic and corporate spending', 'economic recovery', 'eurozone economy', 'foreign capital', 'google', 'growth', 'japan', 'kids', 'public offering', 'rand', 'shell', 'south africa', 'total', 'us dollar', 'us trade deficit', 'value-focused mutual funds'], ['central square in lynn', 'dell', 'gary winnick', 'global crossing', 'government', 'kevin b. rollins', 'large loss', 'oil', 'quality distribution', 'russia', 'stein'], ['e-mail', "internet's killer application"], ['economy', 'financial planner', 'google ipo', 'insurance benefits', 'non-opec oil exporters', 'oil prices', 'opec', 'securities regulation', 'stock market'], ['experimental drugs', 'nitromed inc.'], ['gateway artisan block'], ['google inc.', 'ipo'], ['indian engineers', 'network rail'], ['indicating the economy is improving from a midsummer slump', 'mi

### Create a dataframe for community colors

In [14]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows.append({"node": node, "color": color, "group": group})
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,$36;849.98 trillion,#57dbcc,1
1,assets of the nation's retail money market mut...,#57dbcc,1
2,ability to draw foreign capital,#57db6e,2
3,back-to-school season,#57db6e,2
4,domestic and corporate spending,#57db6e,2
...,...,...,...
59,ultra-cynics,#db8d57,15
60,saudi arabia mobile phone licences,#db577e,16
61,uae's etisalat,#db577e,16
62,short-sellers,#57db9d,17


### Add colors to the graph

In [15]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [16]:
from pyvis.network import Network

graph_output_directory = "./reports/graphs/graph.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory)

                                       
[Select a Node by ID                                                    ]
Reset Selection
